In [2]:
pip install ta talipp arch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 804.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.7/981.7 kB 6.2 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29413 sha256=925d3191f29c531e7a42d5047035468f0a2b3948209b1a914ddbb66a78be3a11
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [3]:
import pandas as pd
import os
import numpy as np
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from datetime import datetime, timedelta
from sklearn.metrics import r2_score
import gzip
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import PredefinedSplit
import xgboost as xgb

import numpy as np
import pandas as pd
import datetime

from sklearn import model_selection

import tensorflow as tf
import tensorflow.keras as keras

from talipp.indicators import *
from talipp.ohlcv import OHLCVFactory
import ta
import arch
from statsmodels.tsa.arima.model import ARIMA

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd

/root


In [7]:
cd ..

/


In [8]:
cd content/drive/MyDrive

/content/drive/MyDrive


In [26]:
data_file = "Historic_Data.parquet.gzip"
data = pd.read_parquet(data_file)

vishesh_file = "Vishesh_Data.parquet.gzip"
vishesh = pd.read_parquet(vishesh_file)

wrds_file = "WRDS_Financial_Data.parquet.gzip"
wrds = pd.read_parquet(wrds_file)

In [27]:
data.replace(-np.inf, np.nan, inplace=True)
data = data.drop(columns=['TICKER','COMNAM','TSYMBOL','NUMTRD','CUSIP'])
data['date'] = pd.to_datetime(data['date'])
data = data.fillna(data.groupby(data['date'].dt.year).transform('median'))

ticker = vishesh.set_index('PERMNO')['TICKER'].to_dict()
data = data.reset_index()
data['TICKER'] = data['PERMNO'].map(ticker)
data = data.drop(columns=['index'])

wrds = wrds.drop(columns=['qdate','public_date','TICKER','cusip','staff_sale'])
wrds['adate'] = pd.to_datetime(wrds['adate'])
wrds['divyield'] = wrds['divyield'].str.rstrip('%').astype('float') / 100.0
wrds = wrds.fillna(wrds.median())
wrds['year'] = wrds['adate'].dt.year
wrds = wrds.drop_duplicates(subset=['permno', 'adate'], keep='first')
wrds = wrds.drop(columns=['adate'])

In [28]:
data['PERMNO'] = data['PERMNO'].astype(float)
data['date'] = pd.to_datetime(data['date'])
duplicate_rows = data[data.duplicated(subset=['PERMNO', 'date'], keep=False)]
data = data.drop_duplicates(subset=['PERMNO', 'date'], keep='first')
data = data.dropna()
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3213291 entries, 0 to 5200378
Data columns (total 13 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   PERMNO   3213291 non-null  float64       
 1   date     3213291 non-null  datetime64[ns]
 2   BIDLO    3213291 non-null  float64       
 3   ASKHI    3213291 non-null  float64       
 4   PRC      3213291 non-null  float64       
 5   VOL      3213291 non-null  float64       
 6   RET      3213291 non-null  float64       
 7   BID      3213291 non-null  float64       
 8   ASK      3213291 non-null  float64       
 9   SHROUT   3213291 non-null  float64       
 10  OPENPRC  3213291 non-null  float64       
 11  RETX     3213291 non-null  float64       
 12  TICKER   3213291 non-null  object        
dtypes: datetime64[ns](1), float64(11), object(1)
memory usage: 343.2+ MB


In [29]:
merged_data = pd.merge(data, wrds, how='left', left_on=['PERMNO', data['date'].dt.year], right_on=['permno', 'year'])
merged_data = merged_data.drop(columns=['permno', 'year'])

In [ ]:
merged_data = merged_data.drop_duplicates(subset=['PERMNO', 'date'], keep='first')
merged_data = merged_data.set_index(['PERMNO', 'date'], drop=True, verify_integrity=True)
data = merged_data
data.info(verbose=True, show_counts=True)

In [32]:
data

BIDLO      ASKHI        PRC        VOL       RET  \
PERMNO  date                                                               
10057.0 1992-01-02    7.00000    7.25000    7.25000    23400.0  0.000000   
        1992-01-03    7.12500    7.25000    7.25000     9100.0  0.000000   
        1992-01-06    7.12500    7.37500    7.37500    15300.0  0.017241   
        1992-01-07    7.37500    7.62500    7.62500    21400.0  0.033898   
        1992-01-08    7.62500    8.75000    8.50000   165500.0  0.114754   
...                       ...        ...        ...        ...       ...   
93096.0 2022-12-23  244.32001  248.78000  247.55000  1452446.0  0.008227   
        2022-12-27  244.64999  248.24500  247.17000  1643906.0 -0.001535   
        2022-12-28  244.77000  249.27499  245.00999  1196778.0 -0.008739   
        2022-12-29  244.99300  249.25500  248.00999   960106.0  0.012244   
        2022-12-30  244.80000  248.63499  246.25000  1145845.0 -0.004879   

                          BID        ASK    SHROUT    OPENPRC      RETX  ...  \
PERMNO  date                                                             ...   
10057.0 1992-01-02   23.50000   24.00000    6291.0   28.75000  0.000000  ...   
        1992-01-03   23.50000   24.00000    6291.0   28.75000  0.000000  ...   
        1992-01-06   23.50000   24.00000    6291.0   28.75000  0.017241  ...   
        1992-01-07   23.50000   24.00000    6291.0   28.75000  0.033898  ...   
        1992-01-08   23.50000   24.00000    6291.0   28.75000  0.114754  ...   
...                       ...        ...       ...        ...       ...  ...   
93096.0 2022-12-23  247.73000  247.74001  225572.0  244.32001  0.008227  ...   
        2022-12-27  247.23000  247.24001  225572.0  247.42999 -0.001535  ...   
        2022-12-28  245.02000  245.10001  225572.0  247.46001 -0.008739  ...   
        2022-12-29  248.05000  248.06000  225572.0  245.64000  0.012244  ...   
        2022-12-30  246.25999  246.31000  223575.0  247.33000 -0.007096  ...   

                   pay_turn  sale_invcap  sale_equity  sale_nwc  rd_sale  \
PERMNO  date                                                               
10057.0 1992-01-02   15.988        2.614        2.752     4.683    0.015   
        1992-01-03   15.988        2.614        2.752     4.683    0.015   
        1992-01-06   15.988        2.614        2.752     4.683    0.015   
        1992-01-07   15.988        2.614        2.752     4.683    0.015   
        1992-01-08   15.988        2.614        2.752     4.683    0.015   
...                     ...          ...          ...       ...      ...   
93096.0 2022-12-23    6.292        1.771        5.465   105.460    0.000   
        2022-12-27    6.292        1.771        5.465   105.460    0.000   
        2022-12-28    6.292        1.771        5.465   105.460    0.000   
        2022-12-29    6.292        1.771        5.465   105.460    0.000   
        2022-12-30    6.292        1.771        5.465   105.460    0.000   

                    adv_sale  accrual    ptb  PEG_trailing  divyield  
PERMNO  date                                                          
10057.0 1992-01-02     0.000   -0.142  0.959         0.785    0.0432  
        1992-01-03     0.000   -0.142  0.959         0.785    0.0432  
        1992-01-06     0.000   -0.142  0.959         0.785    0.0432  
        1992-01-07     0.000   -0.142  0.959         0.785    0.0432  
        1992-01-08     0.000   -0.142  0.959         0.785    0.0432  
...                      ...      ...    ...           ...       ...  
93096.0 2022-12-23     0.003   -0.018  7.189         0.981    0.0237  
        2022-12-27     0.003   -0.018  7.189         0.981    0.0237  
        2022-12-28     0.003   -0.018  7.189         0.981    0.0237  
        2022-12-29     0.003   -0.018  7.189         0.981    0.0237  
        2022-12-30     0.003   -0.018  7.189         0.981    0.0237  

[2551424 rows x 79 columns]

In [33]:
def add_technical_indicators(df):
    grouped = df.groupby('PERMNO')

    for permno, group in grouped:
        # RSI (Relative Strength Index)
        df.loc[permno, 'rsi_14'] = ta.momentum.RSIIndicator(group['PRC'], window=14).rsi()

        # SMA (Simple Moving Average)
        sma_indicator = ta.trend.SMAIndicator(group['PRC'], window=50)
        df.loc[permno, 'sma_50'] = sma_indicator.sma_indicator()
        # Convert 'sma_50' to float
        df['sma_50'] = pd.to_numeric(df['sma_50'], errors='coerce').astype(float)

        # EMA (Exponential Moving Average)
        ema_indicator = ta.trend.EMAIndicator(group['PRC'], window=20)
        df.loc[permno, 'ema_20'] = ema_indicator.ema_indicator()
        # Convert 'ema_20' to float
        df['ema_20'] = pd.to_numeric(df['ema_20'], errors='coerce').astype(float)

        # MACD (Moving Average Convergence Divergence)
        macd_indicator = ta.trend.MACD(group['PRC'], window_slow=26, window_fast=12, window_sign=9)
        df.loc[permno, 'macd'] = macd_indicator.macd()
        df.loc[permno, 'macd_diff'] = macd_indicator.macd_diff()
        df.loc[permno, 'macd_signal'] = macd_indicator.macd_signal()
        # Convert 'macd', 'macd_diff', and 'macd_signal' to float
        df['macd'] = pd.to_numeric(df['macd'], errors='coerce').astype(float)
        df['macd_diff'] = pd.to_numeric(df['macd_diff'], errors='coerce').astype(float)
        df['macd_signal'] = pd.to_numeric(df['macd_signal'], errors='coerce').astype(float)

        # ADX (Average Directional Index)
        adx_indicator = ta.trend.ADXIndicator(group['ASKHI'], group['BIDLO'], group['PRC'], window=14)
        df.loc[permno, 'adx'] = adx_indicator.adx()
        df.loc[permno, 'adx_pos'] = adx_indicator.adx_pos()
        df.loc[permno, 'adx_neg'] = adx_indicator.adx_neg()
        # Convert 'adx', 'adx_pos', and 'adx_neg' to float
        df['adx'] = pd.to_numeric(df['adx'], errors='coerce').astype(float)
        df['adx_pos'] = pd.to_numeric(df['adx_pos'], errors='coerce').astype(float)
        df['adx_neg'] = pd.to_numeric(df['adx_neg'], errors='coerce').astype(float)

        # Stochastic RSI
        stochrsi_indicator = ta.momentum.StochRSIIndicator(group['PRC'], window=14, smooth1=3, smooth2=3)
        df.loc[permno, 'stochrsi'] = stochrsi_indicator.stochrsi()
        df.loc[permno, 'stochrsi_k'] = stochrsi_indicator.stochrsi_k()
        df.loc[permno, 'stochrsi_d'] = stochrsi_indicator.stochrsi_d()
        # Convert 'stochrsi_k' and 'stochrsi_d' to float
        df['stochrsi'] = pd.to_numeric(df['stochrsi'], errors = 'coerce').astype(float)
        df['stochrsi_k'] = pd.to_numeric(df['stochrsi_k'], errors='coerce').astype(float)
        df['stochrsi_d'] = pd.to_numeric(df['stochrsi_d'], errors='coerce').astype(float)

        # Stochastic Oscillator
        stoch_indicator = ta.momentum.StochasticOscillator(group['ASKHI'], group['BIDLO'], group['PRC'], window=14, smooth_window=3)
        df.loc[permno, 'stoch'] = stoch_indicator.stoch()
        df.loc[permno, 'stoch_signal'] = stoch_indicator.stoch_signal()
        # Convert 'stoch' and 'stoch_signal' to float
        df['stoch'] = pd.to_numeric(df['stoch'], errors='coerce').astype(float)
        df['stoch_signal'] = pd.to_numeric(df['stoch_signal'], errors='coerce').astype(float)

        # ROC (Rate of Change)
        roc_indicator = ta.momentum.ROCIndicator(group['PRC'], window=12)
        df.loc[permno, 'roc'] = roc_indicator.roc()
        # Convert 'roc' to float
        df['roc'] = pd.to_numeric(df['roc'], errors='coerce').astype(float)

        # True Strength Index (TSI)
        tsi_indicator = ta.momentum.tsi(group['PRC'], window_slow=25, window_fast=13)
        df.loc[permno, 'tsi'] = tsi_indicator
        # Convert 'tsi' to float
        df['tsi'] = pd.to_numeric(df['tsi'], errors='coerce').astype(float)

        # Ease of Movement (EoM)
        eom_indicator = ta.volume.EaseOfMovementIndicator(group['ASKHI'], group['BIDLO'], group['VOL'], window=14)
        df.loc[permno, 'eom'] = eom_indicator.ease_of_movement()
        df.loc[permno, 'eom_sma'] = eom_indicator.sma_ease_of_movement()
        # Convert 'eom' and 'eom_sma' to float
        df['eom'] = pd.to_numeric(df['eom'], errors='coerce').astype(float)
        df['eom_sma'] = pd.to_numeric(df['eom_sma'], errors='coerce').astype(float)

        # Chaikin Money Flow (CMF)
        cmf_indicator = ta.volume.ChaikinMoneyFlowIndicator(group['ASKHI'], group['BIDLO'], group['PRC'], group['VOL'], window=20)
        df.loc[permno, 'cmf'] = cmf_indicator.chaikin_money_flow()
        df['cmf'] = pd.to_numeric(df['cmf'], errors='coerce').astype(float)

        # Accumulation/Distribution Index (ADI)
        adi_indicator = ta.volume.AccDistIndexIndicator(group['ASKHI'], group['BIDLO'], group['PRC'], group['VOL'])
        df.loc[permno, 'adi'] = adi_indicator.acc_dist_index()
        df['adi'] = pd.to_numeric(df['adi'], errors='coerce').astype(float)

        # Force Index (FI)
        fi_indicator = ta.volume.ForceIndexIndicator(group['PRC'], group['VOL'], window=13)
        df.loc[permno, 'fi'] = fi_indicator.force_index()
        df['fi'] = pd.to_numeric(df['fi'], errors='coerce').astype(float)

        # Money Flow Index (MFI)
        mfi_indicator = ta.volume.MFIIndicator(group['ASKHI'], group['BIDLO'], group['PRC'], group['VOL'], window=14)
        df.loc[permno, 'mfi'] = mfi_indicator.money_flow_index()
        df['mfi'] = pd.to_numeric(df['mfi'], errors='coerce').astype(float)

        # On-Balance Volume (OBV)
        obv_indicator = ta.volume.OnBalanceVolumeIndicator(group['PRC'], group['VOL'])
        df.loc[permno, 'obv'] = obv_indicator.on_balance_volume()
        df['obv'] = pd.to_numeric(df['obv'], errors='coerce').astype(float)

        # Average True Range (ATR)
        atr_indicator = ta.volatility.AverageTrueRange(group['ASKHI'], group['BIDLO'], group['PRC'], window=14)
        df.loc[permno, 'atr'] = atr_indicator.average_true_range()
        df['atr'] = pd.to_numeric(df['atr'], errors='coerce').astype(float)

        # Bollinger Bands
        bollinger_indicator = ta.volatility.BollingerBands(group['PRC'], window=20, window_dev=2)
        df.loc[permno, 'bollinger_hband'] = bollinger_indicator.bollinger_hband()
        df.loc[permno, 'bollinger_hband_indicator'] = bollinger_indicator.bollinger_hband_indicator()
        df.loc[permno, 'bollinger_lband'] = bollinger_indicator.bollinger_lband()
        df.loc[permno, 'bollinger_lband_indicator'] = bollinger_indicator.bollinger_lband_indicator()
        df.loc[permno, 'bollinger_mavg'] = bollinger_indicator.bollinger_mavg()
        df.loc[permno, 'bollinger_pband'] = bollinger_indicator.bollinger_pband()
        df.loc[permno, 'bollinger_wband'] = bollinger_indicator.bollinger_wband()

        df['bollinger_hband'] = pd.to_numeric(df['bollinger_hband'], errors='coerce').astype(float)
        df['bollinger_hband_indicator'] = pd.to_numeric(df['bollinger_hband_indicator'], errors='coerce').astype(float)
        df['bollinger_lband'] = pd.to_numeric(df['bollinger_lband'], errors='coerce').astype(float)
        df['bollinger_lband_indicator'] = pd.to_numeric(df['bollinger_lband_indicator'], errors='coerce').astype(float)
        df['bollinger_mavg'] = pd.to_numeric(df['bollinger_mavg'], errors='coerce').astype(float)
        df['bollinger_pband'] = pd.to_numeric(df['bollinger_pband'], errors='coerce').astype(float)
        df['bollinger_wband'] = pd.to_numeric(df['bollinger_wband'], errors='coerce').astype(float)

    return df

In [34]:
def add_fft(df):
    grouped = df.groupby('PERMNO')

    for permno, group in grouped:
        df.loc[permno, 'fft'] = np.fft.fft(group['PRC'])

    return df

In [35]:
def fit_garch_model(prices, p, q):
    model = arch.arch_model(prices, vol='Garch', p=p, q=q)
    result = model.fit(disp='off')
    return result.conditional_volatility

def addGARCH_of_differentfactors(df):
    # create a series of p and q
    p_values = range(1, 3)
    q_values = range(1, 3)
    grouped = df.groupby('PERMNO')

    for permno, group in grouped:
      for p in p_values:
        for q in q_values:
          df.loc[permno, f'GARCH_{p}_{q}'] = fit_garch_model(group['PRC'], p, q)

    return df

In [36]:
def fit_arma_model(series, ar_order, ma_order):
    arma_model = ARIMA(series, order=(ar_order, 0, ma_order))
    arma_results = arma_model.fit()
    return arma_results.fittedvalues

def add_ARMA(df):
    ar_values = range(1, 3)
    ma_values = range(1, 3)
    grouped = df.groupby('PERMNO')

    for _, group in grouped:
        for ar in ar_values:
            for ma in ma_values:
                arma_col_name = f'ARMA_{ar}_{ma}'
                df.loc[group.index, arma_col_name] = fit_arma_model(group['PRC'], ar, ma)

    return df

In [37]:
data = add_technical_indicators(data)
data = data.dropna()
data = add_fft(data)
data = addGARCH_of_differentfactors(data)

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:765: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


In [43]:
data['fft'] = pd.to_numeric(data['fft'], errors='coerce').astype(float)
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2492541 entries, (10057.0, Timestamp('1992-03-12 00:00:00')) to (93096.0, Timestamp('2022-12-30 00:00:00'))
Data columns (total 115 columns):
 #    Column                     Non-Null Count    Dtype  
---   ------                     --------------    -----  
 0    BIDLO                      2492541 non-null  float64
 1    ASKHI                      2492541 non-null  float64
 2    PRC                        2492541 non-null  float64
 3    VOL                        2492541 non-null  float64
 4    RET                        2492541 non-null  float64
 5    BID                        2492541 non-null  float64
 6    ASK                        2492541 non-null  float64
 7    SHROUT                     2492541 non-null  float64
 8    OPENPRC                    2492541 non-null  float64
 9    RETX                       2492541 non-null  float64
 10   TICKER                     2492541 non-null  object 
 11   CAPEI                      2492541 

In [45]:
# save data to a paraquet.gzip file
data.to_parquet('ML_Project_data.parquet.gzip', compression='gzip')